In [58]:
import numpy as np
from scipy import signal, sparse
%matplotlib inline
import matplotlib.pyplot as plt
import pprint as pp

In [37]:
n = 9
m = 9
N = n * m
tempInitial = 0
initialStates = tempInitial * np.ones([n * m])

In [38]:
# Material Properties
materialHeatCapacity = 840  # J/(kg-K)
materialDensity = 1850  # kg/m^3
symRunTime = 6000  # run time; seconds
materialConductivity = 10  # W/(m-K)
plateLength = 0.16  # side length; meters
plateThickness = 0.18  # meters
powerInput = 190  # Value of total step power input; watts
plateArea = plateLength**2
convectionCoeffTop = 15  # Convection heat loss coefficient on top of plate
convectionCoeffBottom = 2  # Convection heat loss coefficient on bottom of plate
powerLossTop = -convectionCoeffTop * (plateArea/N)  # Value of step power loss top
powerLossBottom = -convectionCoeffBottom * (plateArea/N)  # Value of step power loss bottom

In [39]:
# Time Step and Vector
simSteps = 100
dt = symRunTime/simSteps
timeVector = np.linspace(dt, symRunTime, simSteps)

dx = plateLength / n
nodeCrossSection = dx * plateThickness  # cross sectional area of a node

materialCapacitance = materialHeatCapacity * dx * nodeCrossSection * materialDensity
# thermal capacitance
materialResistance = dx / (materialConductivity * nodeCrossSection)
# thermal resistance

In [59]:
# Setup the state space 'A' Matrix

tempA1 = (-(4*materialResistance**3)/(materialCapacitance*materialResistance**4)+powerLossTop+powerLossBottom)\
        * np.ones([n*m])
# Value of C_i,j for internal elements
tempA1[0:n] = -(3 * materialResistance ** 2) / (materialCapacitance * materialResistance ** 3) + \
        powerLossTop + powerLossBottom
# Value of C_i,j for left edge replacement
tempA1[n*m-n:n*m] = -(3 * materialResistance ** 2) / \
        (materialCapacitance * materialResistance ** 3) + powerLossTop + powerLossBottom
# Value of C_i,j for right edge replacement

# This loop modifies the C_i,j value for edge elements along the top and bottom of the grid by checking
# if they are an even multiple of n or are one larger than an even multiple of n
for value in range(N):
    if value % n == 0:
        tempA1[value] = -(3 * materialResistance ** 2) / (materialCapacitance * materialResistance ** 3)\
            + powerLossTop + powerLossBottom  # bottom row
    elif value % n == (n - 1):
        tempA1[value] = -(3 * materialResistance ** 2) / (materialCapacitance * materialResistance ** 3)\
            + powerLossTop + powerLossBottom  # top row
    else:
        pass

# This block adjusts the corners to the correct coefficient values.
tempA1[0] = -(2 * materialResistance) / (materialCapacitance * materialResistance ** 2) + \
    powerLossTop + powerLossBottom
tempA1[n - 1] = tempA1[0]
tempA1[N - 1] = tempA1[0]
tempA1[n * m - n] = tempA1[0]

# tempA2 will map to two diagonals directly next to the main diagonal and
# represents resistance between C_i,j and C_i,j(+/-)1 (elements east and
# west). The A Matrix is constructed by first going down the left column
# and finishing by going down the right column, so offsetting the diagonal
# by n rows takes care of removing resistances where there is no element to
# the east or west.

tempA2 = (1 / (materialCapacitance * materialResistance)) * np.ones([n * m - n])

# tempA3 will map to a diagonal which represents resistance between C_i,j and
# C_i-1,j (element south)

tempA3 = (1 / (materialCapacitance * materialResistance)) * np.ones([n * m - 1])

# This loop removes elements from tempA3 at the bottom edge of the grid array,
# where there is no thermal element to the south.

for value in range(N - 1):
    if (value + 1) % n == 0:
        tempA3[value] = 0
    else:
        pass

# tempA4 will map to a diagonal which represents resistance between C_i,j and
# C_i+1,j (element north)

tempA4 = (1 / (materialCapacitance * materialResistance)) * np.ones(n * m - 1)

# This loop removes elements from tempA4 at the top edge of the grid array,
# where there is no thermal element to the north.

for value in range(N - 1):
    if (value + 1) % n == 1:
        tempA4[value - 1] = 0
    else:
        pass

# Combine state variable coefficient values into A matrix
v = np.zeros(n)  # zeros required to offset east and west edges
tempAEast = np.concatenate((v, tempA2))  # elements to the east
tempAWest = np.concatenate((tempA2, v))  # elements to the west
tempA3 = np.concatenate((tempA3, [0]))  # elements to the south, offset
tempA4 = np.concatenate(([0], tempA4))  # elements to the north, offset

# Vectors required to create sparse A matrix
data = np.array(([tempAWest], [tempA3], [tempA1], [tempA4], [tempAEast]))
diagonals = np.array([-n, -1, 0, 1, n])
# data = np.array(([tempA1]))

sysA = sparse.diags(data, [-n, -1, 0, 1, n], [N, N])
# sysA = scipy.sparse.diags(data, [0], [N, N])

In [63]:
sysBComponents = np.zeros(N)

sysBComponents[0:n] = 1 / (materialCapacitance * materialResistance)

sysB = sysBComponents

sysUPowerInput = np.zeros_like(timeVector)

sysUPowerInput[0:len(timeVector)] = powerInput / np.count_nonzero(sysBComponents)

sysU = sysUPowerInput

sysC = np.eye(N)

sysD = np.array([0])

sys = np.array((sysA, sysB, sysC, sysD))
print(type(sysA))
print(sysA.shape)
print(type(sysB))
print(sysB.shape)
print(type(sysC))
print(sysC.shape)
print(type(sysD))
print(sysD.shape)

<class 'scipy.sparse.dia.dia_matrix'>
(81, 81)
<class 'numpy.ndarray'>
(81,)
<class 'numpy.ndarray'>
(81, 81)
<class 'numpy.ndarray'>
(1,)


In [64]:
tOut, yOut, xOut = signal.lsim(sys, sysU, timeVector)

ValueError: The input arrays have incompatible shapes.